# Coding a spam classifier with naive Bayes

### 1. Imports and pre-processing data

We load the data into a Turi Create SFrame, and then preprocess it by adding a string with the (non-repeated) words in the email.

In [15]:
import numpy as np

In [16]:
import pandas as pd
emails = pd.read_csv('./emails.csv')

In [17]:
emails[:10]

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
5,"Subject: great nnews hello , welcome to medzo...",1
6,Subject: here ' s a hot play in motion homela...,1
7,Subject: save your money buy getting this thin...,1
8,Subject: undeliverable : home based business f...,1
9,Subject: save your money buy getting this thin...,1


In [18]:
def process_email(text):
    text = text.lower()
    return list(set(text.split()))

emails['words'] = emails['text'].apply(process_email)

In [19]:
emails[:10]

,text,spam,words
0,Subject: naturally irresistible your corporate...,1,"[with, -, a, isoverwhelminq, changes, nowadays..."
1,Subject: the stock trading gunslinger fanny i...,1,"[chisel, try, or, the, hall, but, clockwork, m..."
2,Subject: unbelievable new homes made easy im ...,1,"[,, unconditionally, -, a, rate, ask, from, sh..."
3,Subject: 4 color printing special request add...,1,"[&, ,, -, a, com, ), irwindale, or, ., format,..."
4,"Subject: do not have money , get software cds ...",1,"[along, ,, with, from, finish, ', ., the, ain,..."
5,"Subject: great nnews hello , welcome to medzo...",1,"[,, as, -, a, va, sav, worldwide, medzonline, ..."
6,Subject: here ' s a hot play in motion homela...,1,"[filings, today, 16, assumptions, already, see..."
7,Subject: save your money buy getting this thin...,1,"[with, man, -, a, ., the, tried, aicohol, over..."
8,Subject: undeliverable : home based business f...,1,"[-, a, unknown, name, grownups, mtpi, of, mon,..."
9,Subject: save your money buy getting this thin...,1,"[with, man, -, a, ., the, tried, over, effect,..."


In [20]:
# Calculating the probability that an email is spam
1.0*sum(emails['spam']==1)/len(emails)

0.2388268156424581

### 2. Coding Naive Bayes

We start by counting how many spam and ham emails contain a given word.

We check for the words 'money' and 'easy'.

In [21]:
def count_spam_ham(word):
    email_count = {'spam': 0, 'ham': 0}
    for index, email in emails.iterrows():
        if word in email['words']:
            if email['spam']:
                email_count['spam'] += 1
            else:
                email_count['ham'] += 1
    return email_count

# In case it's a dictionary
'''
def count_spam_ham(word):
    email_count = {'spam': 0, 'ham': 0}
    for email in emails:
        if word in email['word_count']:
            if email['spam']:
                email_count['spam'] += 1
            else:
                email_count['ham'] += 1
    return email_count
'''

"\ndef count_spam_ham(word):\n    email_count = {'spam': 0, 'ham': 0}\n    for email in emails:\n        if word in email['word_count']:\n            if email['spam']:\n                email_count['spam'] += 1\n            else:\n                email_count['ham'] += 1\n    return email_count\n"

In [22]:
print(count_spam_ham('money'))
print(count_spam_ham('easy'))

{'spam': 280, 'ham': 87}
{'spam': 110, 'ham': 61}


Now we make a function that takes a number of words. The naive Bayes algorithm goes over all these words, multiplies the probabilities that the email containing them are spam, and ham. Finally, calculates the weighted probabilities using Naive Bayes, and returns the probability that the email is spam.

In [23]:
def prob_spam_bayes(word):
    # Returns the probability that the email is spam given that it contains a word
    spam, ham = count_spam_ham(word)
    if spam==0 and ham==0:
        return 0.5
    return 1.0*spam/(spam+ham)

In [24]:
def prob_spam_naive_bayes(words):
    email_counts = [count_spam_ham(word) for word in words]
    spams = [count['spam'] for count in email_counts]
    hams = [count['ham'] for count in email_counts]
    #print spams
    #print hams
    spam = np.prod([count['spam'] for count in email_counts])
    ham = np.prod([count['ham'] for count in email_counts])
    if spam==0 and ham==0:
        return 0.5
    return 1.0*spam/(spam+ham)

# In case the email comes as a string
def prob_spam_naive_bayes_string(email):
    words = email.split()
    return prob_spam_naive_bayes(words)

### Testing with some sample emails
We verify that for non-spammy words, the classifier gives us small probabilities, and for spammy words it gives us large probabilities.

In [25]:
prob_spam_naive_bayes(['money', 'easy'])

0.8530201899908605

In [26]:
prob_spam_naive_bayes(['mom','friend','school'])

0.008857887217413228

In [27]:
prob_spam_naive_bayes(['prince','viagra'])

1.0

In [28]:
prob_spam_naive_bayes_string('hi mom how are you please buy apples')

0.0

In [15]:
prob_spam_naive_bayes_string('buy cheap viagra get lottery')

1.0

In [16]:
prob_spam_naive_bayes_string('enter in the lottery now win three million dollars')

1.0

In [17]:
prob_spam_naive_bayes_string('lets meet at the hotel lobby at nine am tomorrow')

0.0

In [18]:
prob_spam_naive_bayes_string('hi mom make easy money')

0.08279582746750283

In [19]:
prob_spam_naive_bayes_string('hi mom')

0.03860711582134747

In [20]:
prob_spam_naive_bayes_string('make easy money')

0.6921082499793675

### 3. Training an efficient model

Our plan is to write a dictionary, and in this dictionary record every word, and its pair of occurrences in spam and ham

In [29]:
model = {}

# Training process
for index, email in emails.iterrows():
    for word in email['words']:
        if word not in model:
            model[word] = {'spam': 1, 'ham': 1}
        if word in model:
            if email['spam']:
                model[word]['spam'] += 1
            else:
                model[word]['ham'] += 1

In [30]:
model

{'with': {'spam': 654, 'ham': 2663},
 '-': {'spam': 998, 'ham': 3534},
 'a': {'spam': 999, 'ham': 3414},
 'isoverwhelminq': {'spam': 8, 'ham': 1},
 'changes': {'spam': 57, 'ham': 253},
 'nowadays': {'spam': 25, 'ham': 2},
 'become': {'spam': 83, 'ham': 112},
 'practicable': {'spam': 25, 'ham': 1},
 'its': {'spam': 186, 'ham': 330},
 'our': {'spam': 623, 'ham': 1492},
 'ordered': {'spam': 37, 'ham': 35},
 'recollect': {'spam': 26, 'ham': 1},
 'stationery': {'spam': 112, 'ham': 1},
 'make': {'spam': 276, 'ham': 711},
 'ciear': {'spam': 10, 'ham': 1},
 'use': {'spam': 207, 'ham': 540},
 'statlonery': {'spam': 16, 'ham': 1},
 'efforts': {'spam': 72, 'ham': 135},
 'of': {'spam': 973, 'ham': 3326},
 'interested': {'spam': 186, 'ham': 457},
 'drafts': {'spam': 25, 'ham': 3},
 'all': {'spam': 472, 'ham': 1206},
 'list': {'spam': 243, 'ham': 400},
 'logo': {'spam': 97, 'ham': 13},
 'break': {'spam': 30, 'ham': 71},
 'gaps': {'spam': 26, 'ham': 8},
 'you': {'spam': 983, 'ham': 3477},
 'stylish':

In [31]:
model['lottery']

{'spam': 9, 'ham': 1}

In [32]:
model['sale']

{'spam': 39, 'ham': 42}

In [40]:
def predict_bayes(word):
    word = word.lower()
    num_spam_with_word = model[word]['spam']
    num_ham_with_word = model[word]['ham']
    return 1.0*num_spam_with_word/(num_spam_with_word + num_ham_with_word)

In [41]:
predict_bayes('lottery')

0.9

In [42]:
predict_bayes('sale')

0.48148148148148145

In [43]:
def predict_naive_bayes(email):
    email = email.lower()
    words = set(email.split())
    spams = []
    hams = []
    for word in words:
        if word in model:
            spams.append(model[word]['spam'])
            hams.append(model[word]['ham'])
    prod_spams = np.long(np.prod(spams))
    prod_hams = np.long(np.prod(hams))
    return 1.0*prod_spams/(prod_spams + prod_hams)

In [44]:
predict_naive_bayes('hi mom how are you')

0.0013894756610580057

In [48]:
predict_naive_bayes('Hi MOM how aRe yoU')

0.0013894756610580057

In [49]:
predict_naive_bayes('enter the lottery to win three million dollars')

0.38569290647197135

In [50]:
predict_naive_bayes('meet me at the lobby of the hotel at nine am')

0.02490194297492509

In [51]:
predict_naive_bayes('buy cheap lottery easy money now')

0.9913514898646872